In [16]:
import pandas as pd
import numpy as np
import math

# Match node puplulation data

In [17]:
column_names = ["From","To","Valid-From","Valid-Until","Days","Departure","Arrival","Flight","Aircraft","Duration"]
data_schedule = pd.read_csv("schedule.txt", sep=";",names=column_names)

#read in the capacity of the aircraft
column_names = ["Name","ICAO","IATA","Capacity","Country"]
data_aircraft = pd.read_csv("aircrafts.txt", sep=";",names=column_names).loc[:, ["Name","ICAO","IATA","Capacity"]]
data_aircraft = data_aircraft.rename(columns={'Name': 'AircraftName'})
data_aircraft = data_aircraft.rename(columns={'IATA': 'Aircraft'})

# read in the airport data
column_names = ["Name","City","Country","IATA","ICAO","Latitude","Longitude","Altitude","Timezone","DST"]
data_airport = pd.read_csv("airports.txt", sep=";",names=column_names).loc[:, ["Name","City","Country","IATA"]]
data_airport = data_airport.rename(columns={'Name': 'Airport'})
# get from airport for merging 
data_airport_from = data_airport.loc[:, ['Airport',"City","Country","IATA"]]
data_airport_from = data_airport_from.rename(columns={'Airport': 'FromAirport','City': 'FromCity','Country': 'FromCountry',"IATA":"From"})
# get to airport for merging 
data_airport_to = data_airport.loc[:, ['Airport',"City","Country","IATA"]]
data_airport_to = data_airport_to.rename(columns={'Airport': 'ToAirport','City': 'ToCity','Country': 'ToCountry',"IATA":"To"})

In [18]:
# repairing the data 

In [19]:
# Loop through the "Valid-From" column of the data_schedule DataFrame
for index, row in enumerate(data_schedule["Valid-From"]):
    # If the value at the current index is '\N'
    if row == '\\N':
        # Set the value at the current index to "2009-01-02"
        data_schedule["Valid-From"][index] = "2009-01-02"

# Loop through the "Valid-Until" column of the data_schedule DataFrame
for index, row in enumerate(data_schedule["Valid-Until"]):
    # If the value at the current index is '\N'
    if row == '\\N':
        # Set the value at the current index to "2009-01-30"
        data_schedule["Valid-Until"][index] = "2009-01-30"

/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [20]:
# 2009-01-05 is Monday, calculate the existing airlines within a week. 2009-01-05 to 2009-01-11

In [21]:
# create a sample dataframe with a date column
df = data_schedule
# convert the date column to datetime
df['Valid-From'] = pd.to_datetime(df['Valid-From'])
# extract the day from the datetime
df['from'] = df['Valid-From'].dt.strftime('%d')
# convert the day to an integer
df['from'] = df['from'].str.extract('(\d+)').astype(int)
df['Valid-Until'] = pd.to_datetime(df['Valid-Until'])
# extract the day from the datetime
df['until'] = df['Valid-Until'].dt.strftime('%d')
# convert the day to an integer
df['until'] = df['until'].str.extract('(\d+)').astype(int)
# filter the dataframe based on the day
#df = df[df['day'] <= 11 and df['day'] >= 5]
df = df.loc[df['until'] >= 12]
data_schedule = df.loc[df['from'] <= 5]

In [22]:
merged_df = pd.merge(data_schedule, data_aircraft, on='Aircraft')
merged_df

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,from,until,AircraftName,ICAO,Capacity
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,2,30,Fairchild Dornier Do.328,D328,33
1,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,3,30,Fairchild Dornier Do.328,D328,33
2,AAL,OSL,2009-01-02,2009-01-30,1234,07:15,08:15,BA8273,D38,01:00,2,30,Fairchild Dornier Do.328,D328,33
3,AAL,OSL,2009-01-02,2009-01-30,17,16:50,17:50,BA8279,D38,01:00,2,30,Fairchild Dornier Do.328,D328,33
4,AAR,OSL,2009-01-03,2009-01-30,2345,16:50,18:00,BA8271,D38,01:10,3,30,Fairchild Dornier Do.328,D328,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,NRT,HNL,2009-01-02,2009-01-30,1234567,21:00,08:30,JO74,747,06:30,2,30,Boeing 747 all pax models,\N,416
9468,NRT,KOA,2009-01-03,2009-01-17,1234567,21:00,08:40,JO70,747,06:40,3,17,Boeing 747 all pax models,\N,416
9469,HEL,TMP,2009-01-02,2009-01-30,6,13:40,14:20,AY3997,AT5,00:40,2,30,Aerospatiale/Alenia ATR 42-500,AT45,50
9470,TMP,HEL,2009-01-02,2009-01-30,6,14:40,15:15,AY3998,AT5,00:35,2,30,Aerospatiale/Alenia ATR 42-500,AT45,50


In [23]:
# list_from_name = []
# list_from_City = []
# list_to_name = []
# for index,from_id in enumerate(merged_df["From"].tolist()): "City"
#     list_from_name.append(data_airport.loc[data_airport['IATA'] == from_id, 'Airport'].values[0])
    
# for index,to_id in enumerate(merged_df["To"].tolist()):
#     list_to_name.append(data_airport.loc[data_airport['IATA'] == to_id, 'Airport'].values[0])

In [24]:
merged_df = pd.merge(merged_df, data_airport_from, on="From")
merged_df = pd.merge(merged_df, data_airport_to, on="To")
merged_df = merged_df.drop_duplicates()



In [25]:
# Within one week flights， what's the total number of passengers 

In [26]:
list_passengers = []
for index, (days,capacity) in enumerate(zip(merged_df['Days'],merged_df["Capacity"])):
    #print(len(str(day))*int(capacity))
    list_passengers.append(int(len(str(days))*int(capacity)/7))
merged_df = merged_df.assign(passenger_pre=list_passengers)
merged_df

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,...,AircraftName,ICAO,Capacity,FromAirport,FromCity,FromCountry,ToAirport,ToCity,ToCountry,passenger_pre
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,...,Fairchild Dornier Do.328,D328,33,Aalborg,Aalborg,Denmark,Aarhus,Aarhus,Denmark,9
1,OSL,AAR,2009-01-03,2009-01-30,2345,18:25,19:35,BA8272,D38,01:10,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18
2,OSL,AAR,2009-01-02,2009-01-30,1234,09:00,10:10,BA8270,D38,01:10,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18
3,OSL,AAR,2009-01-02,2009-01-30,17,18:20,20:05,BA8280,D38,01:45,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,9
4,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,...,Fairchild Dornier Do.328,D328,33,Aalborg,Aalborg,Denmark,Gardermoen,Oslo,Norway,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,SJU,FDF,2009-01-02,2009-01-30,1357,20:20,22:18,AA5020,AT7,01:58,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Lamentin,Fort-de-france,Martinique,42
9468,SJU,NEV,2009-01-02,2009-01-30,1234567,12:35,13:54,AA5086,AT7,01:19,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Vance Winkworth Amory International Airport,\N,Saint Kitts and Nevis,74
9469,SJU,PTP,2009-01-02,2009-01-30,1357,20:30,22:03,AA5012,AT7,01:33,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Raizet,Pointe-a-pitre,Guadeloupe,42
9470,SJU,SLU,2009-01-02,2009-01-30,1234567,20:15,22:18,AA5018,AT7,02:03,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,74


In [33]:
data_city = pd.read_csv("city_dataset/worldcities.csv").loc[:, ["country","city","population"]]
data_city[data_city['city']=='Castries']

,country,city,population
788,Saint Lucia,Castries,70000.0
37674,France,Castries,6250.0


In [27]:
# following is the demo for with data in city Aarhus
data_city = pd.read_csv("city_dataset/worldcities.csv").loc[:, ["city","population"]]
#generate the population of to city and from city
data_city = data_city.rename(columns={'city': 'ToCity'})
#data_city
merged_df_1 = pd.merge(merged_df, data_city, how='inner', on="ToCity")
merged_df_1

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,...,ICAO,Capacity,FromAirport,FromCity,FromCountry,ToAirport,ToCity,ToCountry,passenger_pre,population
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,...,D328,33,Aalborg,Aalborg,Denmark,Aarhus,Aarhus,Denmark,9,237551.0
1,OSL,AAR,2009-01-03,2009-01-30,2345,18:25,19:35,BA8272,D38,01:10,...,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18,237551.0
2,OSL,AAR,2009-01-02,2009-01-30,1234,09:00,10:10,BA8270,D38,01:10,...,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18,237551.0
3,OSL,AAR,2009-01-02,2009-01-30,17,18:20,20:05,BA8280,D38,01:45,...,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,9,237551.0
4,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,...,D328,33,Aalborg,Aalborg,Denmark,Gardermoen,Oslo,Norway,18,693494.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20639,SJU,BON,2009-01-02,2009-01-30,246,20:15,22:19,AA5044,AT7,02:04,...,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Flamingo,Kralendijk,Netherlands Antilles,31,NaN
20640,SJU,SLU,2009-01-02,2009-01-30,1234567,20:15,22:18,AA5018,AT7,02:03,...,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,74,70000.0
20641,SJU,SLU,2009-01-02,2009-01-30,1234567,20:15,22:18,AA5018,AT7,02:03,...,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,74,6250.0
20642,SJU,SLU,2009-01-02,2009-01-30,567,18:40,20:51,AA5080,AT7,02:11,...,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,31,70000.0


In [14]:
data_city

,ToCity,population
0,Tokyo,39105000.0
1,Jakarta,35362000.0
2,Delhi,31870000.0
3,Manila,23971000.0
4,São Paulo,22495000.0
...,...,...
42900,Tukchi,10.0
42901,Numto,10.0
42902,Nord,10.0
42903,Timmiarmiut,10.0


In [15]:
# get the dictionary about city : population 
data_city_list = data_city["FromCity"].tolist()
data_city_population = data_city["population"].tolist()
data_from_list = merged_df["FromCity"].tolist()
data_to_list = merged_df["ToCity"].tolist()
flight_city_list = set(data_from_list+data_to_list)
city_polulation_dict = {}
for city in flight_city_list:
    try:
        popuindex = data_city_list.index(city)
        city_polulation_dict[city] = data_city_population[popuindex]
    except:
        pass
    
#lso get another dictionary for income infected person in cities, initalize it with zero
from_city = merged_df["FromCity"].tolist()
to_city  = merged_df["ToCity"].tolist()
cities_in = list(set(from_city+to_city))
infected_polulation_city = {}
for city_in in cities_in:
    infected_polulation_city[city_in] = 0
    
# Also get another dictionary for income infected person in this term, initalize it with zero
from_airport = merged_df["FromAirport"].tolist()
to_airport = merged_df["ToAirport"].tolist()
airports = list(set(from_airport+to_airport))
infected_polulation_in_airport = {}
for airport in airports:
    infected_polulation_in_airport[airport] = 0
    
    
    
#adding an coreelation airport2city and city2airport：
airport2city = {}
city2airport = {}
for index,(from_airport,from_city,to_airport,to_city) in enumerate(zip(merged_df["FromAirport"].tolist(),merged_df["FromCity"].tolist(),merged_df["ToAirport"].tolist(),merged_df["ToCity"].tolist())):
    airport2city[from_airport] = from_city
    airport2city[to_airport] = to_city
for key, value in airport2city.items():
    city2airport[value] = key

KeyError: 'FromCity'

In [ ]:
# demo about calculatiing the inital infected person.
# demo for if there is x people infected by virus, which is lambda% of total y people.
# Accordingly, the infected person for this flight is merged_df["passengers"] * lambda%

In [ ]:
#if lambda is 3% percentage of people infected in New York, the total infected person is city_polulation_dict["New York"]*0.03
# Then considering all airlines from New York
result_airline = merged_df.loc[merged_df['FromCity'] == "New York"]
# Then we can get  infected person to all destionation  , if infected rate during the travel is 0.4(this need to be calculated by you)
for to_airport, infected_num in zip(result_airline["ToAirport"].tolist(),result_airline["Capacity"].tolist()):
    infected_polulation_in_airport[to_airport]+=int(infected_num)

In [ ]:
infected_polulation_in_airport

In [ ]:
for in_airport in infected_polulation_in_airport.keys():
    city = airport2city[in_airport]
    infected_polulation_city[city] += infected_polulation_in_airport[in_airport]
    

In [ ]:
infected_polulation_city

# SEIR model

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import datetime

In [ ]:
dup_df = merged_df[['From', 'Departure', 'Flight', 'passenger_pre']]

dup_df = dup_df.duplicated(['From', 'Departure', 'Flight'], keep=False)

merged_df = merged_df.assign(dup_flight=dup_df.to_list())
 
def re_cal_passengers(p, dup):
    if dup:
        return math.ceil(p/2)
    else: 
        return p

merged_df['passenger']= merged_df.apply(lambda a: re_cal_passengers(a.passenger_pre, a.dup_flight), axis=1)
merged_df

In [ ]:
merged_df = merged_df.append({'From': 'AAL', 'To':'SPI', 'passenger': 0, 'Duration': '00:00'}, ignore_index=True)
merged_df = merged_df.append({'From': 'AAL', 'To':'KMG', 'passenger': 0, 'Duration': '00:00'}, ignore_index=True)
merged_df = merged_df.append({'From': 'AAL', 'To':'TKN', 'passenger': 0, 'Duration': '00:00'}, ignore_index=True)
merged_df = merged_df.append({'From': 'BJX', 'To':'TKN', 'passenger': 0, 'Duration': '00:00'}, ignore_index=True)

adj = pd.crosstab(merged_df.From, merged_df.To, values=merged_df.passenger, aggfunc='sum').fillna(0)

terminal_name = list(adj.columns.values)
with open(r'./terminal_name.txt', 'w') as fp:
    fp.write('\n'.join(terminal_name))
    
adj_matrix = adj.to_numpy()
np.save("init_flight_capacity.npy", adj_matrix, allow_pickle=True)

In [ ]:
adj

In [ ]:
infected_polulation_in_airport = {}
for airport in airports:
    infected_polulation_in_airport[airport] = 0

to_population = adj.copy()
for col in to_population.columns:
    to_population[col].values[:] = 0
for to_airport, infected_num in zip(result_airline["ToAirport"].tolist(), result_airline["ToAirport"].tolist(),result_airline["Capacity"].tolist()):
#     infected_polulation_in_airport[to_airport]+=int(infected_num)

In [ ]:
# the transmission rate (or the rate of contact)  𝛽
beta = 0.
# The incubation rate 𝜎 is the rate at which exposed hosts become infected, 
sigma = 1
# The recovery rate 𝛾 is the rate at which infected individuals move to the R category
gamma = 0.1
healthy = 0.99



def seir_f(t, y, beta, sigma, gamma):
    s, e, i, r = y
    return np.array([-beta * i * s,
                     -sigma * e + beta * i * s, 
                     -gamma * i + sigma * e, 
                     gamma * i])

num_of_passenger = 300
# infect
# p is the s, e, i , r in the flight
p_f = [healthy, 1-healthy, 0, 0]

# p is the s, e, i , r in the city
p_c = [healthy, 1-healthy, 0.3, 0.3]

def get_infection_ei(p, num_of_people, place_arg):
    beta_, sigma_, gamma_ = place_arg
    sol = solve_ivp(seir_f, [0, 24], p, 
                    rtol=1e-6, args=(beta_, sigma_, gamma_))
    return np.array(sol.y[:, 1])[1:3].sum()*num_of_people

infection_ppl = get_infection_ei(p_f, num_of_passenger, (beta, sigma, gamma))

In [ ]:
from scipy.integrate import odeint
# FUNCTION TO RETURN DERIVATIVES AT T

beta = 1 # infection rate
gamma = 0.1 # recovery rate

def f(y,t):
    S, I, R = y 
    d0 = -beta*S*I # derivative of S(t)
    d1 = beta*S*I - gamma*I # derivative of I(t)
    d2 = gamma*I # derivative of R(t)
    return [d0, d1, d2]

S_0 = 1
I_0 = 50/109_581_078
R_0 = 0

y_0 = [S_0,I_0,R_0]
t = np.linspace(start=0,stop=100,num=100)
y = odeint(f,y_0,t)
# print(y)
S = y[:,0]
I = y[:,1]
R = y[:,2]

plt.plot(t,S,'r',label='S(t)')
plt.plot(t,I,'b',label='I(t)')
plt.plot(t,R,'g',label='R(t)')
plt.legend()
plt.show()

In [ ]:
def duration2minutes(time_duration):
    time = datetime.datetime.strptime(str(time_duration), "%M:%S")

    # Convert the datetime object to the number of seconds
    seconds = time.minute  * 60 + time.second
    return seconds
merged_df['duration_minutes'] = merged_df['Duration'].apply(duration2minutes)
merged_df['duration_minutes'] = merged_df['duration_minutes'].subtract(merged_df['duration_minutes'].min())
merged_df['duration_minutes'] = merged_df['duration_minutes'].divide(merged_df['duration_minutes'].max()-merged_df['duration_minutes'].min())
merged_df['infection_rate'] =  merged_df['duration_minutes']*(1-0.779)+0.779
merged_df['infection_rate'].max()
beta = pd.crosstab(merged_df.From, merged_df.To, values=merged_df.infection_rate, aggfunc='mean').fillna(0)
beta = beta.to_numpy()
np.save("beta.npy", beta, allow_pickle=True)

In [ ]:
from scipy.integrate import odeint

beta = np.load("beta.npy") # infection rate
gamma = np.copy(beta)
gamma[gamma != 0] = 0.03 # recovery rate


In [ ]:
# FUNCTION TO RETURN DERIVATIVES AT T


def SIR(y):
    S, I, R = y 
    d0 = S - beta*S*I # derivative of S(t)
    d1 = I + beta*S*I - gamma*I # derivative of I(t)
    d2 = R+ gamma*I # derivative of R(t)
    
    return [d0, d1, d2]

def loop_infection(n_iter, ):
    
    infection = np.zeros((len(adj_matrix), len(adj_matrix)))
    infection[0][2] = 1
    # print(infection)

    y_0 = (np.nan_to_num((adj_matrix-infection)/adj_matrix,nan = 1),\
           np.nan_to_num(infection/adj_matrix, nan = 0),\
           np.zeros(len(adj_matrix)))
    s, i ,r = SIR(y_0)
    updated_infection_people = np.ceil((i*adj_matrix).sum())
    
    
    return 

print(y_0)

print(s, i, r)
print()

In [ ]:
# Graph = np.array([[0,1,0,0,0],
#                   [0,0,1,0,0],
#                   [1,0,0,0,0],
#                   [0,0.5,0,0,0.5],
#                   [0,0,0,1,0]])
# ini_PR = np.array([0.2,0.2,0.2,0.2,0.2])

Graph = adj_matrix*beta
print(len(Graph))
ini_PR = np.zeros(len(Graph))
ini_PR[0] = 1
ini_PR

In [ ]:
from numpy import linalg as LA

def pagerank(iter_n, d, g, pr,norm):
    for i in range(iter_n):
        dot_product = np.dot(pr_matrix[i],g)
        pr = (1-d)+d*dot_product
pagerank(20,1,Graph,ini_PR,"L1")